# Binary Discrete Choice Models
Fill me in

## Setup

In [7]:
include("Include.jl");

  Activating project at `~/Desktop/julia_work/CHEME-5760-Examples-F23`
    Updating registry at `~/.julia/registries/General.toml`
    Updating git-repo `https://github.com/varnerlab/VLDecisionsPackage.jl.git`
  No Changes to `~/Desktop/julia_work/CHEME-5760-Examples-F23/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-5760-Examples-F23/Manifest.toml`


## Data

In [2]:
dataset = HondaTeslaDataSet()

Row,feature,exponent,Tesla,Honda
,String15,Float64,Float64,Float64
1,sustainability,0.2,5.0,3.0
2,affordability,0.1,2.0,4.0
3,styling,0.05,5.0,2.0
4,usefulness,0.3,2.0,5.0
5,costownership,0.1,4.0,2.0
6,performance,0.05,5.0,1.0
7,safety,0.2,5.0,5.0


### Analytical Logit Choice Model

In [3]:
model = build(VLLinearUtilityFunction, (
        α = dataset[:,:exponent],)
);

In [4]:
V = zeros(2);
V[1] = model(dataset[:,:Tesla]);
V[2] = model(dataset[:,:Honda]);

In [5]:
p = exp(V[1])/(exp(V[1])+exp(V[2]))

0.46257015465625034

In [8]:
d = Bernoulli(p)

Bernoulli{Float64}(p=0.46257015465625034)

In [67]:
M = 10000
S = rand(d,M);

In [68]:
idx = findall(x-> x == 0, S) |> x-> length(x)/M

0.5433

### Monte-Carlo Simulation of Discrete Binary Choice

In [64]:
ϵ = Gumbel(0,π/6)
N = 10000
D₁ = V[1] .+ rand(ϵ,N);
D₂ = V[2] .+ rand(ϵ,N);

In [65]:
D = [D₁ D₂];
choice_vector = Array{Int64,1}();
for i ∈ 1:N
    if (D[i,1] > D[i,2])
        push!(choice_vector,0)
    else
        push!(choice_vector,1)
    end
end

In [66]:
findall(x-> x == 0, choice_vector) |> x-> length(x)/N

0.4283